In [9]:
import whisper

In [10]:
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [05:28<00:00, 4.64MiB/s]


In [25]:
result = model.transcribe("hf_tts_inferences/1.wav")

In [26]:
result

{'text': ' Hello, world, text to speech.',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 1.84,
   'text': ' Hello, world, text to speech.',
   'tokens': [50364, 2425, 11, 1002, 11, 2487, 281, 6218, 13, 50456],
   'temperature': 0.0,
   'avg_logprob': -0.48352666334672406,
   'compression_ratio': 0.7837837837837838,
   'no_speech_prob': 0.021048247814178467}],
 'language': 'en'}

In [14]:
import evaluate

I0430 15:21:41.399141 9837 config.py:58] PyTorch version 2.3.0 available.


In [15]:
bleu_metric = evaluate.load("evaluate-metric/bleu")

In [22]:
predictions = [" How are you?"]
references = ["How are you?"]

In [23]:
bleu_results = bleu_metric.compute(predictions=predictions, references=references)

In [24]:
bleu_results

{'bleu': 1.0,
 'precisions': [1.0, 1.0, 1.0, 1.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 4,
 'reference_length': 4}